In [9]:
#!pip install stable_baselines3
#!pip install yfinance yahoo_fin pandas
#!pip install pyarrow pandas --upgrade
!pip install empyrical

     ---------------------------------------- 0.0/52.8 kB ? eta -:--:--
     -------------- ----------------------- 20.5/52.8 kB 320.0 kB/s eta 0:00:01
     ------------------------------------ - 51.2/52.8 kB 518.5 kB/s eta 0:00:01
     -------------------------------------- 52.8/52.8 kB 538.6 kB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for pandas-datareader>=0.2 from https://files.pythonhosted.org/packages/3f/16/56c9d648b503619ebe96f726b5f642b68e299b34162ed2d6faa9d7966b7d/pandas_datareader-0.10.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/109.5 kB ? eta -:--:--
   -------------------------- ------------- 71.7/109.5 kB 2.0 MB/s eta 0:00:01
   ---------------------------------------- 109.5/109.5 kB 2.1 MB/s eta 0:00:00
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39796 sha256=84ef9019d1d93ec74db8652081ffc4c46bd13580

In [10]:
import gym
from gym import spaces
import numpy as np
import pandas as pd
import empyrical as ep
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv

In [3]:
path = 'D:/Disco/Data/MT5/D1/'

In [5]:
symbols = pd.read_csv('IBXLDia_29-11-24.csv', sep=';', encoding='UTF-8', skiprows=1)['codigo'].index
symbols

Index(['ALOS3', 'ABEV3', 'ASAI3', 'AZUL4', 'AZZA3', 'B3SA3', 'BBSE3', 'BBDC4',
       'BBAS3', 'BRAV3', 'BRFS3', 'BPAC11', 'CMIG4', 'CPLE6', 'CSAN3', 'CYRE3',
       'ELET3', 'EMBR3', 'ENGI11', 'EQTL3', 'GGBR4', 'NTCO3', 'HAPV3', 'HYPE3',
       'ITSA4', 'ITUB4', 'JBSS3', 'KLBN11', 'RENT3', 'LREN3', 'MGLU3', 'MRVE3',
       'MULT3', 'PETR3', 'PETR4', 'PRIO3', 'RADL3', 'RDOR3', 'RAIL3', 'SBSP3',
       'CSNA3', 'SUZB3', 'VIVT3', 'TIMS3', 'TOTS3', 'UGPA3', 'USIM5', 'VALE3',
       'VBBR3', 'WEGE3'],
      dtype='object')

time
2023-10-25    23.28
2023-10-26    23.66
2023-10-27    23.15
2023-10-30    22.60
2023-10-31    22.97
              ...  
2024-10-14    22.43
2024-10-15    22.49
2024-10-16    22.43
2024-10-17    22.51
2024-10-18    22.22
Name: close, Length: 248, dtype: float64

In [ ]:
for symbol in symbols:
    data = pd.read_parquet(f'{path}{symbols[0]}.parquet')
    data['return'] = ep.ann
    data['vol'] = 


In [ ]:

class PortfolioEnv(gym.Env):
    def __init__(self, data, max_assets=4):
        super(PortfolioEnv, self).__init__()
        self.data = data
        self.max_assets = max_assets
        self.num_assets = data.shape[1] - 2  # Supondo que as 2 primeiras colunas são retorno e volatilidade
        self.current_step = 0
        # Ação: selecionar até max_assets ativos
        self.action_space = spaces.MultiBinary(self.num_assets)
        # Estado: retorno, volatilidade e portfólio atual
        self.observation_space = spaces.Box(low=0, high=np.inf, 
                                            shape=(self.num_assets * 2 + self.num_assets,), 
                                            dtype=np.float32)
    
    def reset(self):
        self.current_step = 0
        self.portfolio = np.zeros(self.num_assets)
        return self._get_obs()
    
    def _get_obs(self):
        returns = self.data.iloc[self.current_step, :self.num_assets].values
        volatility = self.data.iloc[self.current_step, self.num_assets:2*self.num_assets].values
        portfolio = self.portfolio
        return np.concatenate([returns, volatility, portfolio])
    
    def step(self, action):
        # Aplicar ação com restrição de max_assets
        if np.sum(action) > self.max_assets:
            reward = -10  # Penalidade por violar a restrição
            done = True
            return self._get_obs(), reward, done, {}
        self.portfolio = action
        # Calcular retorno do portfólio
        returns = self.data.iloc[self.current_step, :self.num_assets].values
        portfolio_return = np.dot(self.portfolio, returns)
        reward = portfolio_return  # Objetivo de maximizar o retorno
        self.current_step += 1
        done = self.current_step >= len(self.data) - 1
        return self._get_obs(), reward, done, {}
    
# Carregar e pré-processar os dados
# data deve ser um DataFrame onde as primeiras 50 colunas são retornos e as próximas 50 são volatilidades
data = pd.read_csv('dados_portfolio.csv')  # Substitua pelo caminho dos seus dados

env = PortfolioEnv(data)
env = DummyVecEnv([lambda: env])

model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100000)

# Salvar o modelo
model.save("ppo_portfolio")

# Carregar o modelo
# model = PPO.load("ppo_portfolio")

# Testar o modelo
obs = env.reset()
for _ in range(len(data)):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    if done:
        break


In [ ]:
path = 'D:/Disco/Data/MT5/D1/'



In [ ]:
from yahoo_fin import stock_info as si

# Obter os ativos mais ativos do dia
most_active = si.get_day_most_active()

# Selecionar os top 50 ativos
top_50 = most_active.head(50)

# Exibir a lista de ativos
print(top_50[['Symbol', 'Volume']])